# Index

以下の論文にしたがって実装を進めていきます。  
https://www.researchgate.net/publication/227658748_Classification_and_Regression_Trees

## 0. 準備

In [127]:
import numpy as np
from collections import Counter

In [128]:
# 特徴量: X_sample
# 列0: 数値特徴量
# 列1: カテゴリ特徴量 (0: 'A', 1: 'B', 2: 'C' と仮定)
X_sample = np.array([
    [1, 0], [2, 1], [3, 0], [4, 2], [5, 1],
    [6, 2], [7, 0], [8, 0], [9, 1], [10, 2]
])
# クラスラベル: y_sample
y_sample = np.array([0, 1, 0, 1, 0, 1, 0, 0, 1, 1])

# 特徴量のタイプ ('numeric' or 'categorical')
feature_types_sample = ['numeric', 'categorical']
# カテゴリ特徴量のマッピング (デバッグや表示用)
category_mapping_sample = {0: 'A', 1: 'B', 2: 'C'}


print("サンプルデータ (X_sample_np):")
print(X_sample)
print("\nサンプルクラスラベル (y_sample_np):")
print(y_sample)
print("\n特徴量タイプ:")
print(feature_types_sample)

サンプルデータ (X_sample_np):
[[ 1  0]
 [ 2  1]
 [ 3  0]
 [ 4  2]
 [ 5  1]
 [ 6  2]
 [ 7  0]
 [ 8  0]
 [ 9  1]
 [10  2]]

サンプルクラスラベル (y_sample_np):
[0 1 0 1 0 1 0 0 1 1]

特徴量タイプ:
['numeric', 'categorical']


## 1. Gini不純度の計算


*   **ステップ 1.1: Gini不純度の概念説明**
    *   決定木は、データをより「純粋な」サブセットに分割していくことで構築されます。純粋とは、サブセット内のクラスラベルがほぼ同じであることを意味します。
    *   Gini不純度は、ノード内のデータがどれだけ混ざっているかを示す指標の一つです。
    *   計算式: $Gini(D) = 1 - Σ (p_k)^2$
        *   $D$: データセット（またはノード内のデータ）
        *   $p_k$: クラス $k$ に属するサンプルの割合
    *   Gini不純度の値の範囲は 0 から 0.5 (2クラスの場合) または最大で '1 - 1/num_classes' です。
        *   0 の場合: ノードは完全に純粋（すべてのサンプルが同じクラス）。
        *   値が大きいほど: ノードの不純度が高い（クラスが混在している）。
    *   論文のCARTのセクションでは、Gini Indexが不純度関数として使われることが言及されています。

*   **ステップ 1.2: `calculate_gini` 関数の実装**
    *   **目的**: 与えられたクラスラベルのリスト（またはpandas Series）からGini不純度を計算します。
    *   **引数**:
        *   `y_subset` (pd.Series or np.array): クラスラベルのサブセット。
    *   **戻り値**:
        *   `imprity`: 計算されたGini不純度。

In [129]:
def calculate_gini(y_subset) -> float:
    # 空のサブセットは不純度 = 0
    if len(y_subset) == 0:
        return 0
    
    # 各クラスのサンプル数をカウント
    counts = Counter(y_subset)
    imprity = 1.0
    for key in counts:
        # クラスkの確率
        p_k = counts[key] / len(y_subset)
        # Gini不純度の計算
        imprity -= p_k ** 2
    return imprity

In [130]:
test_labels1 = np.array([0, 1, 0, 1, 0, 1, 0, 0, 1, 1])
test_labels2 = np.array([0, 0, 1, 1, 0, 1, 1, 1, 1, 1])
test_labels3 = np.array([0, 0, 0, 1, 0, 1, 0, 1, 0, 1])
print("\nGini impurity for test_labels1:", calculate_gini(test_labels1))
print("Gini impurity for test_labels2:", calculate_gini(test_labels2))
print("Gini impurity for test_labels3:", calculate_gini(test_labels3))


Gini impurity for test_labels1: 0.5
Gini impurity for test_labels2: 0.4200000000000001
Gini impurity for test_labels3: 0.48


## 2. 最適な分割の探索


*   **ステップ 2.1: 分割の概念説明**
    *   決定木は、各ノードでデータセットを2つの子ノードに分割します。
    *   最適な分割とは、分割後の子ノードの不純度（Gini不純度の加重平均）が最も小さくなるような分割です。これは、親ノードの不純度からの「Gini不純度の減少量 (Gini Reduction)」が最大になる分割とも言えます。
    *   `Gini Reduction = Gini(parent) - (weight_left * Gini(left_child) + weight_right * Gini(right_child))`
    *   数値特徴量の場合: `X <= threshold` のような条件で分割します。全てのユニークな値の間の中点が分割候補となります。
    *   カテゴリ特徴量の場合: `X == category_value` (True/False) のような条件で分割します。各ユニークなカテゴリ値が分割候補となります。(論文のTHAIDはより一般的にサブセットSを探索します。)

*   **ステップ 2.2: `find_best_split` 関数の実装**
    *   **目的**: 与えられたデータサブセットに対して、すべての特徴量とすべての可能な分割点を試し、Gini不純度を最も減少させる分割を見つけます。
    *   **引数**:
        *   `X_subset`: 特徴量のサブセット。
        *   `y_subset`: クラスラベルのサブセット。
        *   `feature_types`: 各特徴量のタイプ ('numeric' or 'categorical')。
    *   **戻り値**:
        *   `best_split_info`: 最適な分割情報 (例: `{'feature_index': ..., 'threshold': ..., 'type': 'numeric', 'gini_reduction': ..., 'left_indices': ..., 'right_indices': ...}`)
        *   分割によって不純度が減少しない、または分割できない場合は `None`。

In [131]:
def find_best_split(X_subset, y_subset, feature_types) -> dict:
    '''
    最適な分割特徴量と分割点を見つける関数
    '''

    best_split_info = None
    max_gini_reduction = -1.0

    parent_gini = calculate_gini(y_subset)
    n_samples = len(y_subset)

    if n_samples <= 1:
        return None
    
    for feature_index in range(X_subset.shape[1]):
        feature_values = X_subset[:, feature_index]
        unique_values = np.unique(feature_values)

        if feature_types[feature_index] == 'numeric':
            sorted_unique_values = np.sort(unique_values)
            threshold_candidates = []

            if len(sorted_unique_values) > 1:
                threshold_candidates = (sorted_unique_values[:-1] + sorted_unique_values[1:]) / 2

            for threshold in threshold_candidates:
                # ブールインデックスを作成
                left_bool_indices = feature_values <= threshold
                right_bool_indices = feature_values > threshold

                y_left = y_subset[left_bool_indices]
                y_right = y_subset[right_bool_indices]

                if len(y_left) == 0 or len(y_right) == 0:
                    continue

                gini_left = calculate_gini(y_left)
                gini_right = calculate_gini(y_right)

                weighted_gini_children = (
                    (len(y_left) / n_samples) * gini_left +
                    (len(y_right) / n_samples) * gini_right
                )

                current_gini_reduction = parent_gini - weighted_gini_children

                if current_gini_reduction > max_gini_reduction:
                    max_gini_reduction = current_gini_reduction
                    best_split_info = {
                        'feature_index': feature_index,
                        'threshold': threshold,
                        'type': 'numeric',
                        'gini_reduction': current_gini_reduction,
                        'left_indices_bool': left_bool_indices,
                        'right_indices_bool': right_bool_indices
                    }

        elif feature_types[feature_index] == 'categorical':
            for category in unique_values:
                left_bool_indices = feature_values == category
                right_bool_indices = feature_values != category
                
                y_left = y_subset[left_bool_indices]
                y_right = y_subset[right_bool_indices]

                if len(y_left) == 0 or len(y_right) == 0:
                    continue

                gini_left = calculate_gini(y_left)
                gini_right = calculate_gini(y_right)

                weighted_gini_children = (
                    (len(y_left) / n_samples) * gini_left +
                    (len(y_right) / n_samples) * gini_right
                )

                current_gini_reduction = parent_gini - weighted_gini_children

                if current_gini_reduction > max_gini_reduction:
                    max_gini_reduction = current_gini_reduction
                    best_split_info = {
                        'feature_index': feature_index,
                        'category': category,
                        'type': 'categorical',
                        'gini_reduction': current_gini_reduction,
                        'left_indices_bool': left_bool_indices,
                        'right_indices_bool': right_bool_indices
                    }
    
    if max_gini_reduction <= 0:
        return None
    
    return best_split_info

In [132]:
# テスト
split_result = find_best_split(X_sample, y_sample, feature_types_sample)
if split_result:
    fest_idx = split_result['feature_index']
    print("\n最適な分割特徴量のインデックス:", fest_idx)
    if split_result['type'] == 'numeric':
        print("分割点:", split_result['threshold'])
    else:
        cat_val = split_result['category']
        cat_name = category_mapping_sample[cat_val]
        print("カテゴリ特徴量の値:", cat_name)
    print("Gini不純度の減少:", split_result['gini_reduction'])
    print("左のインデックスブール:", split_result['left_indices_bool'])
    print("右のインデックスブール:", split_result['right_indices_bool'])

else:
    print("\n最適な分割が見つかりませんでした。")


最適な分割特徴量のインデックス: 1
カテゴリ特徴量の値: A
Gini不純度の減少: 0.33333333333333337
左のインデックスブール: [ True False  True False False False  True  True False False]
右のインデックスブール: [False  True False  True  True  True False False  True  True]


## 3. 決定木のノード構造

*   **ステップ 3.1: `Node` クラスの概念説明**
    *   決定木はノードの集まりで構成されます。各ノードは以下の情報を持つことができます。
        *   **中間ノード (Internal Node)**: データを分割するための条件（特徴量、閾値/カテゴリ）。左右の子ノードへの参照。
        *   **葉ノード (Leaf Node)**: 最終的な予測クラスラベル。
    *   また、デバッグや分析のために、ノードのGini不純度や含まれるサンプル数などの情報も保持しておくと便利です。
*   **ステップ 3.2: `Node` クラスの実装**
    *   **目的**: 決定木の各ノードを表すクラスを定義します。

In [133]:
class Node:
    def __init__(
            self,
            feature_index=None,
            threshold=None,
            category=None,
            feature_type=None,
            left_child=None,
            right_child=None,
            value=None,
            gini=None,
            num_samples=None,
    ):
        self.feature_index = feature_index
        self.threshold = threshold
        self.category = category
        self.feature_type = feature_type
        self.left_child = left_child
        self.right_child = right_child
        self.value = value
        self.gini = gini
        self.num_samples = num_samples

    def is_leaf_node(self):
        return self.value is not None    

In [134]:
leaf_node_example = Node(value=0, gini=0.0, num_samples=10)
print(f"leaf_node_exampleは葉ノードですか？: {leaf_node_example.is_leaf_node()}")
print(f"leaf_node_exampleの値: {leaf_node_example.value}, Gini: {leaf_node_example.gini}, サンプル数: {leaf_node_example.num_samples}")

leaf_node_exampleは葉ノードですか？: True
leaf_node_exampleの値: 0, Gini: 0.0, サンプル数: 10


## 4. 決定木の構築

*   **ステップ 4.1: `_build_tree` メソッドの概念説明 (ClassificationTreeクラス内)**
    *   このメソッドは、決定木を再帰的に構築する主要なロジックを含みます。
    *   **処理の流れ**:
        1.  **停止条件のチェック**:
            *   現在のノードの深さが `max_depth` に達したか？
            *   現在のノードのサンプル数が `min_samples_split` より少ないか？
            *   現在のノードのクラスラベルがすべて同じか (Gini不純度が0か)？
            *   もし停止条件を満たせば、葉ノードを作成して終了。葉ノードの値は、そのノード内のサンプルの多数決で決定します。
        2.  **最適な分割の探索**: `find_best_split` を呼び出して、現在のデータセットに対する最適な分割を見つけます。
        3.  **分割不可の場合**: もし有益な分割が見つからなければ（例: Gini Reductionが0以下）、葉ノードを作成して終了。
        4.  **葉ノードの最小サンプル数チェック**: 分割によって生成される子ノードのサンプル数が `min_samples_leaf` を下回る場合、分割せずに葉ノードを作成します。
        5.  **再帰的な分割**:
            *   見つかった最適な分割に基づいて、データセットを左の子ノード用と右の子ノード用に分割します。
            *   左の子ノードと右の子ノードに対して、`_build_tree` を再帰的に呼び出します。
            *   返された左右の子ノードを使って、現在のノードを作成します。
    *   論文の Algorithm 1 "Pseudocode for tree construction by exhaustive search" の Step 3 "If a stopping criterion is reached, exit. Otherwise, apply step 2 to each child node in turn." がこの再帰構造に対応します。

*   **ステップ 4.2: `ClassificationTree` クラスと `_build_tree` メソッドの実装**
    *   **目的**: 分類木全体を管理するクラスと、その中で木を再帰的に構築するメソッドを実装します。

In [135]:
class ClassificationTree:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.root = None
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.feature_types = None

    def _majority_vote(self, y_data):
        '''
        最頻のクラスラベルを返す
        '''
        if len(y_data) == 0:
            return None
        counts = Counter(y_data)
        return counts.most_common(1)[0][0]
    
    def _build_tree(self, X_data, y_data, depth=0):
        '''
        決定木を再帰的に構築する
        '''
        n_samples, n_features = X_data.shape
        current_gini = calculate_gini(y_data)
        
        # 停止条件のチェック
        if(
            (self.max_depth is not None and depth >= self.max_depth) or
            (n_samples < self.min_samples_split) or
            (len(np.unique(y_data)) == 1) 
        ):
            leaf_value = self._majority_vote(y_data)
            return Node(value=leaf_value, gini=current_gini, num_samples=n_samples)
        
        best_split_info = find_best_split(X_data, y_data, self.feature_types)

        # 最適な分割が見つからない場合は葉ノードを返す
        if best_split_info is None:
            leaf_value = self._majority_vote(y_data)
            return Node(value=leaf_value, gini=current_gini, num_samples=n_samples)
        
        left_indices_bool = best_split_info['left_indices_bool']
        right_indices_bool = best_split_info['right_indices_bool']

        # min_samples_leafを下回る場合は葉ノードを返す
        if(
            np.sum(left_indices_bool) < self.min_samples_leaf or
            np.sum(right_indices_bool) < self.min_samples_leaf
        ):
            leaf_value = self._majority_vote(y_data)
            return Node(value=leaf_value, gini=current_gini, num_samples=n_samples)
        
        # 再帰的に左の子ノードを構築
        left_node = self._build_tree(
            X_data[left_indices_bool],
            y_data[left_indices_bool],
            depth + 1
        )

        # 再帰的に右の子ノードを構築
        right_node = self._build_tree(
            X_data[right_indices_bool],
            y_data[right_indices_bool],
            depth + 1
        )

        node_params = {
            'feature_index': best_split_info['feature_index'],
            'feature_type': best_split_info['type'],
            'left_child': left_node,
            'right_child': right_node,
            'gini': current_gini,
            'num_samples': n_samples
        }

        if best_split_info['type'] == 'numeric':
            node_params['threshold'] = best_split_info['threshold']
        else:
            node_params['category'] = best_split_info['category']

        return Node(**node_params)
    
    def fit(self, X_train, y_train, feature_types_list):
        self.feature_types = feature_types_list
        self.root = self._build_tree(X_train, y_train)
        print("決定木が構築されました。")

In [136]:
tree_model_test = ClassificationTree(max_depth=2)
tree_model_test.fit(X_sample, y_sample, feature_types_sample)
print("エラーが発生しませんでした。決定木モデルが正常に構築されました。")

決定木が構築されました。
エラーが発生しませんでした。決定木モデルが正常に構築されました。


## 5. 予測処理

*   新しいデータサンプルが与えられたとき、構築された決定木の根ノードから葉ノードまでをたどり、予測クラスを見つけます。
*   **処理の流れ**:
    1.  現在のノードが葉ノードなら、そのノードの予測値を返します。
    2.  現在のノードが中間ノードなら、そのノードの分割条件（特徴量と閾値/カテゴリ）を使って、データサンプルが左の子ノードに進むべきか、右の子ノードに進むべきかを判断します。
    3.  選ばれた子ノードに対して、`_traverse_tree` を再帰的に呼び出します。

*   **ステップ 5.2: `_traverse_tree` および `predict` メソッドの実装**
    *   **目的**: `_traverse_tree` は単一サンプルの予測を行い、`predict` は複数のサンプルに対して予測を行います。

In [137]:
class ClassficationTree(ClassificationTree): # 継承
    def _traverse_tree(self, X_sample, node): # X_sampleは1D Numpy
        if node.is_leaf_node():
            return node.value
        
        feature_val = X_sample[node.feature_index]

        if node.feature_type == 'numeric':
            if feature_val <= node.threshold:
                return self._traverse_tree(X_sample, node.left_child)
            else:
                return self._traverse_tree(X_sample, node.right_child)
        elif node.feature_type == 'categorical':
            if feature_val == node.category:
                return self._traverse_tree(X_sample, node.left_child)
            else:
                return self._traverse_tree(X_sample, node.right_child)
        else:
            raise ValueError("Unknown feature type: {}".format(node.feature_type))
        
    def predict(self, X_test): # X_testは2D Numpy
        if self.root is None:
            raise ValueError("The model has not been trained yet.")
        
        predictions = []
        for i in range(X_test.shape[0]):
            row_sample = X_test[i, :]
            predictions.append(self._traverse_tree(row_sample, self.root))
        return np.array(predictions)        

In [138]:
# テスト
tree_model_test_for_predict = ClassficationTree(max_depth=3)
tree_model_test_for_predict.fit(X_sample, y_sample, feature_types_sample)

if tree_model_test_for_predict.root:
    sample_prediction = tree_model_test_for_predict.predict(X_sample)
    print("\nサンプルデータの予測結果:")
    print(sample_prediction)
    print("オリジナルのクラスラベル:")
    print(y_sample)
else:
    print("\n決定木モデルが正しく構築されていません。予測を行うことができません。")

決定木が構築されました。

サンプルデータの予測結果:
[0 1 0 1 0 1 0 0 1 1]
オリジナルのクラスラベル:
[0 1 0 1 0 1 0 0 1 1]


## 6. ツリーの可視化

*   **ステップ 6.1: ツリー表示関数の概念説明**
    *   構築された決定木を視覚的に確認できると、モデルの理解が深まります。
    *   ここでは、テキストベースで木の構造を簡易的に表示する関数を作成します。
*   **ステップ 6.2: `print_tree` 関数の実装**
    *   **目的**: 構築されたツリーを再帰的にたどり、各ノードの情報を表示します。

In [139]:
def print_tree(
        node,
        depth=0,
        feature_names=None,
        class_names=None,
        category_names=None,
        indent_char='  '
):
    if node is None:
        print(f'{indent_char * depth}Empty node')
        return
    
    indent = indent_char * depth

    if node.is_leaf_node():
        class_name = str(node.value)
        if class_names and node.value in class_names:
            class_name = class_names[node.value]
        print(f'{indent}Leaf: {class_name} (Gini: {node.gini:.4f}, Samples: {node.num_samples})')
        return
    
    feature_name = f'Feature_{node.feature_index}'
    if feature_names and node.feature_index < len(feature_names):
        feature_name = feature_names[node.feature_index]

    if node.feature_type == 'numeric':
        condition = f'{feature_name} <= {node.threshold:.4f}'
        condition_else = f'{feature_name} > {node.threshold:.4f}'
    else:
        cat_code = node.category
        cat_display_name = str(cat_code)
        if category_names and node.feature_index in category_names and cat_code in category_names[node.feature_index]:
            cat_display_name = category_names[node.feature_index][cat_code]

        condition = f'{feature_name} == {cat_display_name}'
        condition_else = f'{feature_name} != {cat_display_name}'

    print(f'{indent}If {condition} (Gini: {node.gini:.4f}, Samples: {node.num_samples})')
    print_tree(node.left_child, depth + 1, feature_names, class_names, category_names, indent_char)
    print(f'{indent}Else {condition_else}')
    print_tree(node.right_child, depth + 1, feature_names, class_names, category_names, indent_char)

In [140]:
# テスト
print("\n決定木の構造:")
if tree_model_test_for_predict.root:
    sample_feature_names = ['Numeric Feature', 'Categorical Feature']
    sample_class_names = {0: 'Class A', 1: 'Class B'}
    sample_category_map = {
        1: category_mapping_sample
    }

    print_tree(tree_model_test_for_predict.root,
                    feature_names=sample_feature_names,
                    class_names=sample_class_names,
                    category_names=sample_category_map)
else:
    print("決定木モデルが正しく構築されていません。ツリーの表示ができません。")


決定木の構造:
If Categorical Feature == A (Gini: 0.5000, Samples: 10)
  Leaf: Class A (Gini: 0.0000, Samples: 4)
Else Categorical Feature != A
  If Numeric Feature <= 5.5000 (Gini: 0.2778, Samples: 6)
    If Numeric Feature <= 4.5000 (Gini: 0.4444, Samples: 3)
      Leaf: Class B (Gini: 0.0000, Samples: 2)
    Else Numeric Feature > 4.5000
      Leaf: Class A (Gini: 0.0000, Samples: 1)
  Else Numeric Feature > 5.5000
    Leaf: Class B (Gini: 0.0000, Samples: 3)


### 7. モデルの学習と評価

In [141]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
X_iris = iris.data
y_iris = iris.target
iris_feature_names = iris.feature_names

# IrisデータセットはすべてNumeric
feature_types_iris = ['numeric'] * X_iris.shape[1]

# データセットの分割
X_train, X_test, y_train, y_test = train_test_split(
    X_iris, y_iris, test_size=0.5, random_state=0, stratify=y_iris
)

# モデルのインスタンス化と学習
iris_tree_model = ClassficationTree(max_depth=3, min_samples_split=5, min_samples_leaf=2)
iris_tree_model.fit(X_train, y_train, feature_types_iris)

# ツリーの表示
print("\nIrisデータセットの決定木の構造:")
iris_class = {0: 'setosa', 1: 'versicolor', 2: 'virginica'}
print_tree(
    iris_tree_model.root,
    feature_names=iris_feature_names,
    class_names=iris_class,
)

# 予測と精度の評価
if iris_tree_model.root:
    test_pred = iris_tree_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, test_pred)
    print("\nテストデータの予測精度:", test_accuracy)

    train_pred = iris_tree_model.predict(X_train)
    train_accuracy = accuracy_score(y_train, train_pred)
    print("学習データの予測精度:", train_accuracy)

決定木が構築されました。

Irisデータセットの決定木の構造:
If petal length (cm) <= 2.5000 (Gini: 0.6667, Samples: 75)
  Leaf: setosa (Gini: 0.0000, Samples: 25)
Else petal length (cm) > 2.5000
  If petal width (cm) <= 1.6500 (Gini: 0.5000, Samples: 50)
    Leaf: versicolor (Gini: 0.0740, Samples: 26)
  Else petal width (cm) > 1.6500
    Leaf: virginica (Gini: 0.0000, Samples: 24)

テストデータの予測精度: 0.9333333333333333
学習データの予測精度: 0.9866666666666667
